<a href="https://colab.research.google.com/github/shankartmv/LLM/blob/main/MyExperiments_llamaindex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install llama-index llama-index-vector-stores-qdrant llama-index-llms-azure-openai langchain sentence-transformers llama-index-embeddings-huggingface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 631.0/631.0 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 67.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 92.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 68.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.0/284.0 kB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This

In [ ]:
!pip install -U qdrant-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.3/206.3 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 9.5 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.14.0 requires protobuf<4.21,>=3.20.3, but you have protobuf 4.25.3 which is incompatible.


In [ ]:
!mkdir -p /content/pdf_data
!mkdir -p /content/vector_db
!cd /content/pdf_data ; wget https://tile.loc.gov/storage-services/master/gdc/gdcdatasets/2020445568/2020445568.zip ; unzip 2020445568.zip; unzip lcwa_gov_pdf_data.zip

--2024-02-16 15:56:14--  https://tile.loc.gov/storage-services/master/gdc/gdcdatasets/2020445568/2020445568.zip
Resolving tile.loc.gov (tile.loc.gov)... 104.16.54.16, 104.16.55.16, 2606:4700::6810:3710, ...
Connecting to tile.loc.gov (tile.loc.gov)|104.16.54.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 706045342 (673M) [application/zip]
Saving to: ‘2020445568.zip’

2020445568.zip      100%[===================>] 673.34M   991KB/s    in 11m 59s 

2024-02-16 16:08:13 (959 KB/s) - ‘2020445568.zip’ saved [706045342/706045342]

Archive:  2020445568.zip
  inflating: lcwa_gov_pdf_data.zip   
  inflating: lcwa_gov_pdf_metadata.csv  
  inflating: README.txt              
Archive:  lcwa_gov_pdf_data.zip
 extracting: bagit.txt               
  inflating: manifest-sha256.txt     
  inflating: manifest-sha512.txt     
  inflating: tagmanifest-sha256.txt  
  inflating: bag-info.txt            
   creating: data/
  inflating: data/MZRWQI7HD6SSDT2AQTG7FN73WTASIU6X.pdf  


In [1]:
from llama_index.core import Document,VectorStoreIndex,SimpleDirectoryReader,ServiceContext
from llama_index.vector_stores.qdrant import QdrantVectorStore
from qdrant_client import QdrantClient, models
import secrets
from llama_index.llms.openai import OpenAI
from llama_index.llms.azure_openai import AzureOpenAI
#from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.extractors import TitleExtractor
from llama_index.core.ingestion import IngestionPipeline
from llama_index.vector_stores.qdrant import QdrantVectorStore

In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-small-en-v1.5"
# set True to compute cosine similarity
bge_embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs={'device': 'cpu'}
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
documents=SimpleDirectoryReader(input_dir="/content/pdf_data/data").load_data(show_progress=True,num_workers=2)

In [5]:
print(f"Type:{type(documents)}\nLength:{len(documents)}\n\n\n")
for i in range(10):
  print(documents[i],"\n---------------------------------------\n")
  #print(secrets.choice(documents),"\n---------------------------------------\n")
pdf_document=Document(text="\n\n".join(doc.text for doc in documents))
print(f"Type:{type(pdf_document)}\n{dir(pdf_document)}\n\n")

Type:<class 'list'>
Length:12556



Doc ID: 933e65a3-8ba0-4b08-a67e-4cd5afea05f1
Text: 792-93 93-94 94-95 95-96 96-97 97-98 98-99 99-00 00-01
01-02231,668236,774240,448243,097245,252 244,403 244,623 245,142
245,694 246,405 050,000100,000150,000200,000250,000Idaho Public School
Fall Membership Fall enrollment as of the last Friday in September
Fall enrollment as of the first Friday in November 
---------------------------------------

Doc ID: 8e11ac87-3436-496e-b0e4-95183180ad86
Text: Direct observations of N 2O5reactivity on ambient aerosol
particles Timothy H. Bertram,1,2Joel A. Thornton,1Theran P.
Riedel,3Ann M. Middlebrook,4 Roya Bahreini,4,5Timothy S.
Bates,6Patricia K. Quinn,6and Derek J. Coffman6 Received 28 July 2009;
accepted 10 September 2009; published 8 October 2009. [1]N
2O5reactivity has been measured directly f... 
---------------------------------------

Doc ID: c33a3352-97ee-4722-a67c-1a23d3807a43
Text: Pacific Marine Environmental Laboratory, 10 km NE of downtown
Seatt

In [6]:
llm = AzureOpenAI(engine="gpt-35-turbo",
                      model="gpt-35-turbo",
                      api_key="XXXXXX",
                      azure_endpoint="URL",
                      api_version="XXXXX" ,
                      temperature=0,
                      top_p=0.2
                      )
service_context = ServiceContext.from_defaults(
    llm=llm, embed_model="local:BAAI/bge-small-en-v1.5"
)

<ipython-input-6-f59452173946>:9: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(


In [7]:
qdrnt_client = QdrantClient(path="/content/vector_db")

In [8]:
qdrnt_vector_store=QdrantVectorStore(client=qdrnt_client,collection_name="1000_pdf_repo",batch_size=25,enable_hybrid=True)
pipeline = IngestionPipeline(
    transformations=[
        SentenceSplitter(chunk_size=1024, chunk_overlap=20),
        TitleExtractor(llm=llm),
        bge_embeddings,
    ],
    vector_store=qdrnt_vector_store,
)
pipeline.run(documents=[pdf_document],show_progress=True)

pytorch_model.bin:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/417 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/699 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/17.7M [00:00<?, ?B/s]

ValidationError: 1 validation error for IngestionPipeline
transformations -> 2
  Can't instantiate abstract class TransformComponent with abstract method __call__ (type=type_error)